In [1]:
# Использовать решатели:
# glpk
# gurobi
# xpress
# gams
# mpec_minlp
# gdpopt.enumerate
# mindtpy.ecp
# cbc
# conopt
# copt
# CPLEX
# ilogcp
# loqo
# minos

In [2]:
from generator_d import GeneratorD
from generator_e import GeneratorE
from generator_p import GeneratorP
from generator_t import GeneratorT
from model_builder import ModelBuilder
from solve_action import SolveAction

In [3]:
from ipywidgets import IntProgress
from IPython.display import display

In [4]:
import xpress
inited = xpress.init('/Users/dambr/venv/lib/python3.12/site-packages/xpress/license/community-xpauth.xpr')

In [5]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
import pyomo.environ as pyo

In [6]:
M = 10**6

In [7]:
e = 1
n = 5
m = 4
k = 3

In [8]:
T = np.array([
    [  1,   0,   0,   0],
    [0.5, 0.5,   0,   0],
    [  0, 0.5, 0.5,   0],
    [  0,   0, 0.5, 0.5],
    [  0,   0,   0,   1]
])

In [9]:
D = np.array([
    [0, 0, 0, 0],
    [0, 0, 1, 1],
    [0, 0, 0, 0],
    [0, 0, 0, 0]
])

In [10]:
P = np.array([
    [1, 1, 1, 1, 1],
    [1, 1, 1, 1, 1],
    [1, 1, 1, 1, 1],
    [1, 1, 1, 1, 1],
    [1, 1, 1, 1, 1]
])

In [11]:
E = np.array([
    [1, 0, 0, 0, 0]
])

In [12]:
model_builder = ModelBuilder(M, k, T, D, P, E)
model = model_builder.build()

In [13]:
def solve(model, solver_name):
    values = []
    solve_action = SolveAction(model, solver_name)
    count = 100
    progress = IntProgress(min=0, max=count, value=0, description=solver_name, bar_style='success')
    display(progress)
    for i in range(count):
        solve_result = solve_action.solve()
        instance = solve_result.instance
        value = pyo.value(instance.OBJ)
        values.append(value)
        progress.value += 1
    return set(values)

In [14]:
solve_action = SolveAction(model, 'glpk')
solve_result = solve_action.solve()
instance = solve_result.instance
instance.A.pprint()

A : Size=12, Index={0, 1, 2, 3}*{0, 1, 2}
    Key    : Lower : Value : Upper : Fixed : Stale : Domain
    (0, 0) :     0 :   1.0 :     1 : False : False : Binary
    (0, 1) :     0 :   0.0 :     1 : False : False : Binary
    (0, 2) :     0 :   0.0 :     1 : False : False : Binary
    (1, 0) :     0 :   0.0 :     1 : False : False : Binary
    (1, 1) :     0 :   0.0 :     1 : False : False : Binary
    (1, 2) :     0 :   1.0 :     1 : False : False : Binary
    (2, 0) :     0 :   0.0 :     1 : False : False : Binary
    (2, 1) :     0 :   0.0 :     1 : False : False : Binary
    (2, 2) :     0 :   1.0 :     1 : False : False : Binary
    (3, 0) :     0 :   0.0 :     1 : False : False : Binary
    (3, 1) :     0 :   0.0 :     1 : False : False : Binary
    (3, 2) :     0 :   1.0 :     1 : False : False : Binary


In [15]:
instance.A.extract_values()

{(0, 0): 1.0,
 (0, 1): 0.0,
 (0, 2): 0.0,
 (1, 0): 0.0,
 (1, 1): 0.0,
 (1, 2): 1.0,
 (2, 0): 0.0,
 (2, 1): 0.0,
 (2, 2): 1.0,
 (3, 0): 0.0,
 (3, 1): 0.0,
 (3, 2): 1.0}

In [16]:
def to_mtx_with_values(A):
    values = A.extract_values()
    (m, k) = np.shape(A)
    result = np.zeros((m, k))
    for i in range(m):
        for j in range(k):
            result[i][j] = values[(i, j)]
    return result

In [17]:
f_in = lambda x: 0 if x == 0 else 1

In [18]:
f_im = lambda x: 0 if x < 1 else 1

In [19]:
f_dep = lambda F, D, x: np.dot(F, D) if x == 0 else np.dot(f_dep(F, D, x - 1), D)

In [32]:
def calculate_result_cost(T, D, P, E, A):
    A = to_mtx_with_values(A)
    m = np.shape(A)[0]
    result = 0
    for requirements in E:
        
        files = np.dot(requirements, T)
        print('Полезные файлы', files)
        
        files_with_dependencies = files + sum([f_dep(files, D, i) for i in range(m)])
        print('Полезные файлы с разрешенными зависимостями', files_with_dependencies)
        
        plugins = np.dot(files_with_dependencies, A)
        print('Поставляемые плагины', plugins)
        
        files_in_delivery = np.dot(A, plugins)
        files_in_delivery = np.array([f_in(x) for x in files_in_delivery])
        print('Поставляемые файлы', files_in_delivery)
        
        requirements_in_delivery = np.dot(T, files_in_delivery)
        requirements_in_delivery = np.array([f_im(x) for x in requirements_in_delivery])
        print('Реализованные требования', requirements_in_delivery)
        
        prices = np.dot(P, requirements_in_delivery)
        print('Цены', prices)
        
        cost = sum(prices)
        print('Стоимость:', cost)
        
        result += cost
    return result

In [33]:
calculate_result_cost(T, D, P, E, instance.A)

Полезные файлы [1. 0. 0. 0.]
Полезные файлы с разрешенными зависимостями [1. 0. 0. 0.]
Поставляемые плагины [1. 0. 0.]
Поставляемые файлы [1 0 0 0]
Реализованные требования [1 0 0 0 0]
Цены [1 1 1 1 1]
Стоимость: 5


5

In [66]:
mtx1 = np.array([
    [0, 2, 0],
    [0, 0, 1],
    [1, 0, 0]
])

In [67]:
mtx2 = np.array([1, 1, 0])

In [68]:
np.dot(mtx1, mtx2)

array([2, 0, 1])

In [69]:
np.dot(np.dot(mtx1, mtx2), mtx2)

2